<a href="https://colab.research.google.com/github/pushpraj-vulcan/pilot-project/blob/main/NDVI_gif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import geemap.geemap as geemap
import ee
import os

ee.Authenticate()
ee.Initialize(project = 'ee-pushprajsingh')
vis_map=geemap.Map()
base=geemap.Map()
base

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [2]:
vis_map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
National_Boundary = ee.FeatureCollection('projects/ee-pushprajsingh/assets/National_Boundary')
base.addLayer(National_Boundary, {}, 'National Boundary')
base

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [4]:
region = ee.Geometry.Polygon(
    [[[67.060547, 4.740675],
     [67.060547, 38.065392],
     [98.964844, 38.065392],
     [98.964844, 4.740675],
     [67.060547, 4.740675]]],
    None,
    False)

In [5]:
ndvi = ee.ImageCollection("MODIS/061/MOD13A2").select('NDVI')
base

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [6]:
ndvi_india = ndvi.map(lambda img: img.set('doy',
ee.Date(img.get('system:time_start')).getRelative('day', 'year')));

In [7]:
distinctDOY = ndvi_india.filterDate('2010-02-14', '2011-02-14');

In [8]:
filter = ee.Filter.equals(leftField='doy',rightField='doy')
join=ee.Join.saveAll('doy_matches')
joincol= ee.ImageCollection(join.apply(distinctDOY,ndvi_india,filter))


In [9]:
comp = joincol.map(lambda img: ee.ImageCollection.fromImages(img.get('doy_matches')).reduce(ee.Reducer.mean()).set('doy',ee.Number(img.get('doy'))))

In [10]:
visparams= {
  'min': 0,
  'max': 9000,
  'palette': [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
  ],
};

In [11]:
rgbVis = comp.map(lambda img: img.visualize(bands=['NDVI_mean'],**visparams).clip(National_Boundary))

In [12]:
 gifParams = {
  'region': region,
  'dimensions': 600,
  'crs': 'EPSG:3857',
  'framesPerSecond': 5,
  'format':'gif'
};


In [13]:
print(rgbVis.getVideoThumbURL(gifParams))

https://earthengine.googleapis.com/v1/projects/ee-pushprajsingh/videoThumbnails/9d5ef943930dd574e1d39c7d99521ab2-2470a9b4689f2e778a876115a9f09d90:getPixels
